# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [165]:
# Norvig's spell checker

import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word))) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Let's import n-gram data

In [166]:
from tqdm import tqdm

bigram = {}
trigram = {}
fourgram = {}
fivegram = {}

with open('bigrams.txt', encoding='latin-1') as f:
    for line in tqdm(f):
        list = line.split()
        # update the bigram dictionary with the bigram and its count
        if tuple(list[1:]) not in bigram:
            bigram[tuple(list[1:])] = int(list[0])
        else:
            bigram[tuple(list[1:])] += int(list[0])

with open('fivegrams.txt', encoding='latin-1') as f:
    for line in tqdm(f):
        list = line.split()
        # update the fivegram dictionary with the fivegram and its count
        if tuple(list[1:]) not in fivegram:
            fivegram[tuple(list[1:])] = int(list[0])
        else:
            fivegram[tuple(list[1:])] += int(list[0])

        # update the 4-gram dictionary with the fourgram and its count
        if tuple(list[1:5]) not in fourgram:
            fourgram[tuple(list[1:5])] = int(list[0])
        else:
            fourgram[tuple(list[1:5])] += int(list[0])
        
        if tuple(list[2:]) not in fourgram:
            fourgram[tuple(list[2:])] = int(list[0])
        else:
            fourgram[tuple(list[2:])] += int(list[0])
        
        # update the 3-gram dictionary with the trigram and its count
        if tuple(list[1:4]) not in trigram:
            trigram[tuple(list[1:4])] = int(list[0])
        else:
            trigram[tuple(list[1:4])] += int(list[0])
        
        if tuple(list[2:5]) not in trigram:
            trigram[tuple(list[2:5])] = int(list[0])
        else:
            trigram[tuple(list[2:5])] += int(list[0])
        
        if tuple(list[3:]) not in trigram:
            trigram[tuple(list[3:])] = int(list[0])
        else:
            trigram[tuple(list[3:])] += int(list[0])

1020385it [00:01, 731122.79it/s]
1044268it [00:09, 114786.77it/s]


In [184]:
def find_word_in_ngram(term, text):
    """
    finds occurrences of a term within a context window of varying sizes in a given text

    args:
        term (str): the term to search for
        text (str): the text to search within

    returns:
        dict: a dictionary where keys represent the length of the context window
              and values represent the count of occurrences of the term within that window
    """

    # nested function to choose the appropriate n-gram based on length
    def choose_ngram(length):
        if length == 1:
            return bigram
        elif length == 2:
            return trigram
        elif length == 3:
            return fourgram
        return fivegram

    # slice the last characters of the text
    end_slice = text[-5:]
    result = {}
    
    # iterate over possible ngram lengths
    for length in range(1, len(end_slice)):
        # choose the appropriate ngram
        ngrams = choose_ngram(length)
        
        # slice the context from the end slice
        context_slice = end_slice[-length:]
        key = tuple(context_slice + [term])
        
        # if the ngram exists in dictionary
        if key in ngrams:
            # if found, add its count to the result dictionary
            result[length] = result.get(length, 0) + ngrams[key]
    
    return result

In [168]:
def find_correct(contexts):
    """
    finds the index of the context with the maximum length and maximum count

    args:
        contexts (list): a list of tuples where each tuple contains a context dictionary and its corresponding value

    returns:
        tuple: a tuple containing the index of the context with the maximum length and maximum count, and its value
    """

    max_context_index = 0
    max_length = 0
    max_count = 0
    max_context = 0

    # iterate through each context and its corresponding value in the list
    for i, (context, value) in enumerate(contexts):
        # check if the current context is not empty and if its length or count is greater than the maximum
        if context and (len(context) > max_length or (len(context) == max_length and context[max(context.keys())] > max_count)):
            # update the variables to reflect the new maximum context
            max_context_index = i
            max_length = len(context)
            max_count = context[max(context.keys())]
            max_context = value

    return max_context_index, max_context

In [169]:
from tqdm import tqdm

def ngram_correction(word, context):
    """
    this function performs correction on a given word within a context using n-gram models

    parameters:
    - word (str): The word to be corrected
    - context (list): The context within which the word occurs
    - verbose (bool): Flag to enable/disable verbose output

    returns:
    - corrected_word (str): the corrected version of the word
    """
    # candidate corrections for the given word from Norvig's spell checker
    cands = candidates(word)
    # print(cands)

    # list to store found contexts for each candidate correction
    contexts = []

    for c in cands:
        # find occurrences of the candidate correction in n-grams within the context
        contexts.append((find_word_in_ngram(c, context), c))

    # if no contexts are found for any candidate correction, choose the most probable correction
    no_context_found = all(len(context) == 0 for context, _ in contexts)

    if no_context_found:  # return if no context is found
        return max(cands, key=P)

    return find_correct(contexts)[1] # 0th index is the index of the context, 1st index is the context itself

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

**Ngram Dataset Selection**:
   The implementation utilizes n-gram datasets, including bigrams, trigrams, fourgrams, and fivegrams. Trigrams and fourgrams were constructed by using fivegrams dataset. These datasets capture the context of words within a given text corpus, allowing for context-aware spelling correction. The choice of using multiple n-gram datasets enables the correction algorithm to consider different levels of context, from immediate neighboring words to more distant word associations.

**Edit Distance Weights**:
   The weights for edit1 (one edit away), edit2 (two edits away), and absent words probabilities are implicitly handled by the underlying implementation of Norvig's spell checker. The `candidates()` function generates possible corrections based on these edit distances, with `known()` and `edits1()` functions filtering candidates based on their presence in the vocabulary or their edit distance of one from the original word. This approach allows for a balance between considering closely related words and exploring further alternatives.

**Context Search**:
   The `find_word_in_ngram()` function is responsible for finding occurrences of a term within a context window of varying sizes in a given text. This function iterates over possible n-gram lengths and searches for the term within the corresponding n-gram dataset. By considering different context window lengths, the algorithm captures both local and global context information, enabling more accurate spelling correction decisions.

**Correction Selection**:
   After finding contexts for each candidate correction, the algorithm selects the correction with the maximum length and maximum count within the context. This selection strategy aims to prioritize corrections that are supported by a larger and more frequent context, improving the overall accuracy of spelling correction.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [170]:
import random
import string
import numpy as np
import pandas as pd

def mutate_word(word):
    """
    mutates a word by randomly inserting, deleting, or changing a letter

    args:
    word (str): the input word to be mutated

    returns:
    str: the mutated word
    """
    mutation_type = random.choice(["insert", "delete", "change"])
    index = random.randint(0, len(word) - 1)  # Adjusted to ensure index is within the word's length
    
    if mutation_type == "insert":
        letter = random.choice(string.ascii_lowercase)
        word = word[:index] + letter + word[index:]
    elif mutation_type == "delete":
        if len(word) > 1 and index > 0:  # Ensure index is at least 1 to avoid empty string after deletion
            word = word[:index] + word[index+1:]
    else:  # change
        letter = random.choice(string.ascii_lowercase)
        while word[index] == letter:
            letter = random.choice(string.ascii_lowercase)
        word = word[:index] + letter + word[index+1:]

    return word


Let's download kaggle dataset https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews?select=test.csv

In [171]:
# test_data = pd.read_csv('test.csv',)
# test_data.columns=['id', 'title', 'text']
# test_data = test_data.dropna()["text"].tolist()[:10000]
# np.savetxt('test.txt', test_data, fmt='%s')

In [172]:
# for i in range(len(test_data)):
#     test_data[i] = test_data[i].split()
#     test_data[i] = test_data[i][:6]

In [173]:
# import string

# def remove_punctuation(text):
#     punctuation = string.punctuation
#     cleaned_text = text.translate(str.maketrans('', '', punctuation))
#     return cleaned_text

# for i in range(len(test_data)):
#     for j in range(len(test_data[i])):
#         test_data[i][j] = remove_punctuation(test_data[i][j])

# np.savetxt('test.txt', test_data, fmt='%s')

In [174]:
test_data = open('test.txt').read().splitlines()

In [182]:
for i in range(len(test_data)):
    test_data[i] = test_data[i].split()

In [185]:
import json

answers_ngram = []

correct = 0
for i in tqdm(range(len(test_data))):
    context = test_data[i]
    word = context[-1].lower()
    context = context[:-1]
    incorrect = mutate_word(word).lower()
    corrected_word = ngram_correction(incorrect, context)
    if word == corrected_word:
        correct += 1

    answers_ngram.append({"correct word": word, 
                          "incorrect word": incorrect, 
                          "corrected word": corrected_word, 
                          "context": context,
                          "is correct": word == corrected_word})

accuracy = correct / len(test_data)
print(f"Accuracy: {accuracy}") # accuracy of ngram algorithm
with open("answer_ngram.json", "w") as file:
    json.dump(answers_ngram, file, indent=4)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:51<00:00, 193.91it/s]

Accuracy: 0.7467


In [186]:
answers_ngram[:5]

[{'correct word': 'have',
  'incorrect word': 'have',
  'corrected word': 'have',
  'context': ['Despite', 'the', 'fact', 'that', 'I'],
  'is correct': True},
 {'correct word': 'jul',
  'incorrect word': 'pjul',
  'corrected word': 'paul',
  'context': ['I', 'bought', 'this', 'charger', 'in'],
  'is correct': False},
 {'correct word': 'their',
  'incorrect word': 'thefr',
  'corrected word': 'their',
  'context': ['Check', 'out', 'Maha', 'Energys', 'website'],
  'is correct': True},
 {'correct word': 'the',
  'incorrect word': 'tpe',
  'corrected word': 'the',
  'context': ['Reviewed', 'quite', 'a', 'bit', 'of'],
  'is correct': True},
 {'correct word': 'incorrect',
  'incorrect word': 'inmorrect',
  'corrected word': 'incorrect',
  'context': ['I', 'also', 'began', 'having', 'the'],
  'is correct': True}]

In [115]:
answers_norvig = []

correct = 0
for i in tqdm(range(len(test_data))):
    context = test_data[i]
    word = context[-1].lower()
    context = context[:-1]
    incorrect = mutate_word(word).lower()
    corrected_word = correction(incorrect)
    if word == corrected_word:
        correct += 1
    answers_norvig.append({"correct word": word, 
                            "incorrect word": incorrect, 
                            "corrected word": corrected_word, 
                            "context": context,
                            "is correct": word == corrected_word})

print(f"Accuracy: {correct/len(test_data)}") # accuracy of Norvig's algorithm

with open("answer_norvig.json", "w") as file:
    json.dump(answers_norvig, file, indent=4)

100%|██████████| 10000/10000 [00:50<00:00, 198.02it/s]


Accuracy: 0.6576


In [213]:
answers_norvig[:5]

[{'correct word': 'has',
  'incorrect word': 'hgs',
  'corrected word': 'his',
  'context': ['One', 'of', 'the', 'other', 'reviewers'],
  'is correct': False},
 {'correct word': 'br',
  'incorrect word': 'bfr',
  'corrected word': 'bar',
  'context': ['A', 'wonderful', 'little', 'production', 'br'],
  'is correct': False},
 {'correct word': 'wonderful',
  'incorrect word': 'woderful',
  'corrected word': 'wonderful',
  'context': ['I', 'thought', 'this', 'was', 'a'],
  'is correct': True},
 {'correct word': 'a',
  'incorrect word': 'ja',
  'corrected word': 'a',
  'context': ['Basically', 'theres', 'a', 'family', 'where'],
  'is correct': True},
 {'correct word': 'time',
  'incorrect word': 'time',
  'corrected word': 'time',
  'context': ['Petter', 'Matteis', 'Love', 'in', 'the'],
  'is correct': True}]

## Test on provided reviews.csv

In [153]:
test_df = pd.read_csv("reviews.csv")

In [206]:
test_array = np.array(test_df.review).tolist()

In [207]:
for i in range(len(test_array)):
    test_array[i] = test_array[i].split()
    test_array[i] = test_array[i][:6]

In [208]:
import string

def remove_punctuation(text):
    punctuation = string.punctuation
    cleaned_text = text.translate(str.maketrans('', '', punctuation))
    return cleaned_text

for i in range(len(test_array)):
    for j in range(len(test_array[i])):
        test_array[i][j] = remove_punctuation(test_array[i][j])

In [211]:
test_array = test_array[:10000]

### Check Norvig's solution

In [212]:
answers_norvig = []

correct = 0
for i in tqdm(range(len(test_array))):
    context = test_array[i]
    word = context[-1].lower()
    context = context[:-1]
    incorrect = mutate_word(word).lower()
    corrected_word = correction(incorrect)
    if word == corrected_word:
        correct += 1
    answers_norvig.append({"correct word": word, 
                            "incorrect word": incorrect, 
                            "corrected word": corrected_word, 
                            "context": context,
                            "is correct": word == corrected_word})

print(f"Accuracy: {correct/len(test_array)}") # accuracy of Norvig's algorithm

with open("answer_norvig.json", "w") as file:
    json.dump(answers_norvig, file, indent=4)

100%|██████████| 10000/10000 [00:58<00:00, 172.17it/s]


Accuracy: 0.6547


## Check my solution

In [214]:
import json

answers_ngram = []

correct = 0
for i in tqdm(range(len(test_array))):
    context = test_array[i]
    word = context[-1].lower()
    context = context[:-1]
    incorrect = mutate_word(word).lower()
    corrected_word = ngram_correction(incorrect, context)
    if word == corrected_word:
        correct += 1

    answers_ngram.append({"correct word": word, 
                          "incorrect word": incorrect, 
                          "corrected word": corrected_word, 
                          "context": context,
                          "is correct": word == corrected_word})

accuracy = correct / len(test_array)
print(f"Accuracy: {accuracy}") # accuracy of ngram algorithm
with open("answer_ngram.json", "w") as file:
    json.dump(answers_ngram, file, indent=4)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:55<00:00, 181.36it/s]


Accuracy: 0.7395


Finally, the number of correctly predicted corrections increased using context from ngrams. Moreover, ngram_corrector runs in about the same time (1 second faster), which is also one of the advantages of the algorithm I wrote. You can check the results in answers_ngram.json and answers_norvig.json. By looking at them, it is possible to determine that words are indeed better defined where context is important. 